# Q&A with the Tavily Web Search API

This notebook demonstrates how to interact with the [Tavily](https://tavily.com/) web search API using Amazon Bedrock Agents.

## 1. Setup

In [ ]:
!python3 -m pip install --upgrade -q botocore boto3 awscli

In [ ]:
import boto3
import botocore
import json
import logging
import uuid

# getting boto3 clients for required AWS services
session = boto3.session.Session(profile_name="bloyal-Admin", region_name="us-east-1")
sts_client = session.client("sts")
iam_client = session.client("iam")
lambda_client = session.client("lambda")
bedrock_agent_client = session.client("bedrock-agent")
bedrock_agent_runtime_client = session.client("bedrock-agent-runtime")
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
print(f"{region=}\n{account_id=}")

## 2. Define Helper Functions

In [ ]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

def get_hcls_agent(
    name_starts_with: str, bedrock_agent_client: botocore.client
) -> list[dict]:
    """
    Get the list of agents whose name starts with the given string.
    """
    agent_info = [
        {"name": agent["agentName"], "id": agent["agentId"]}
        for agent in bedrock_agent_client.list_agents().get("agentSummaries")
        if agent["agentName"].startswith("web-search-agent")
    ]
    return agent_info


def get_agent_alias(agent_id: str, bedrock_agent_client: botocore.client) -> list[dict]:
    """
    Get the list of aliases for the given agent.
    """
    alias_info = [
        {"name": alias["agentAliasName"], "id": alias["agentAliasId"]}
        # alias
        for alias in bedrock_agent_client.list_agent_aliases(agentId=agent_id).get(
            "agentAliasSummaries"
        )
    ]
    return alias_info

def invoke_agent_helper(
    query, session_id, agent_id, alias_id, enable_trace=False, session_state=None, end_session=False
):
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8').strip()
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace']['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



## 3. Get Agent Details

In [ ]:
available_agents = get_hcls_agent("web-search-agent", bedrock_agent_client)
current_agent = available_agents[0]
print(f"Current agent name: {current_agent['name']}")
print(f"Current agent id: {current_agent['id']}")

current_agent_alias = get_agent_alias(current_agent['id'], bedrock_agent_client)[0]
print(f"Current agent alias: {current_agent_alias['id']}")

session_id:str = str(uuid.uuid1())
agent_id = current_agent['id']
agent_alias_id = current_agent_alias['id']

## 4. Chat with Agent

In [ ]:
invoke_agent_helper("Who is the current CEO of Pfizer?", session_id, agent_id, agent_alias_id, enable_trace=False)

In [ ]:
invoke_agent_helper("Who won Super Bowl LIX?", session_id, agent_id, agent_alias_id, enable_trace=False)

In [ ]:
invoke_agent_helper("What was the closing stock price of AMZN on March 25, 2024?", session_id, agent_id, agent_alias_id, enable_trace=False)